![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [11]:
import pandas as pd
import numpy as np

pd.set_option("display.width", 1000)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

# For lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Run OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

As with most data science projects, you will need to pre-process the data provided, in this case, a csv file called `rental_info.csv`. Specifically, you need to:

- Read in the csv file `rental_info.csv` using `pandas`.

In [12]:
df_rental = pd.read_csv("rental_info.csv")

print(df_rental.head())

                 rental_date                return_date  amount  release_year  rental_rate  length  replacement_cost                special_features  NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_2
0  2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99        2005.0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}      0   0      0  1    8.9401   15876.0         8.9401
1  2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99        2005.0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}      0   0      0  1    8.9401   15876.0         8.9401
2  2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99        2005.0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}      0   0      0  1    8.9401   15876.0         8.9401
3  2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99        2005.0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}      0   0      0  1    8.9401 

- Create a column named `"rental_length_days"` using the columns `"return_date"` and `"rental_date"`, and add it to the `pandas` DataFrame.     
    This column should contain information on how many days a DVD has been rented by a customer.

In [13]:
df_rental["rental_length"] = pd.to_datetime(df_rental["return_date"]) - pd.to_datetime(df_rental["rental_date"])
df_rental["rental_length_days"] = df_rental["rental_length"].dt.days

- Create two columns of dummy variables from `"special_features"`, which takes the value of 1 when:
    - The value is `"Deleted Scenes"`, storing as a column called `"deleted_scenes"`.
    - The value is `"Behind the Scenes"`, storing as a column called `"behind_the_scenes"`.

In [14]:
# Add dummy for deleted scenes
df_rental["deleted_scenes"] =  np.where(df_rental["special_features"].str.contains("Deleted Scenes"), 1, 0)
# Add dummy for behind the scenes
df_rental["behind_the_scenes"] =  np.where(df_rental["special_features"].str.contains("Behind the Scenes"), 1, 0)

- Make a `pandas` DataFrame called `X` containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.

In [15]:
# Choose columns to drop
cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

# Split into feature and target sets
X = df_rental.drop(cols_to_drop, axis=1)

- Choose the `"rental_length_days"` as the target column and save it as a pandas Series called `y`.

In [16]:
y = df_rental["rental_length_days"]

Following the preprocessing you will need to:

- Split the data into `X_train`, `y_train`, `X_test`, and `y_test` train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.
- **Set `random_state` to `9`** whenever you use a function/method involving randomness, for example, when doing a test-train split.

In [17]:
# Further split into training and test data
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=9)

**Recommend a model yielding a mean squared error (MSE) less than 3 on the test set**

Save the model you would recommend as a variable named `best_model`, and save its MSE on the test set as `best_mse`.

In [ ]:
lr = LinearRegression()
lr = lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)
mse_lr = mean_squared_error(y_test, y_test_pred)

print(mse_lr)

2.941723864697602


In [ ]:
param_dist = {'n_estimators': np.arange(1,101,1), 'max_depth':np.arange(1,11,1)}

rf = RandomForestRegressor()

rand_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, random_state=9)

rand_search.fit(X_train, y_train)

hyper_params = rand_search.best_params_

rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"],  max_depth=hyper_params["max_depth"], random_state=9)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test, rf_pred)

print(mse_random_forest)

2.225667528098759


In [20]:
# Random forest gives lowest MSE so:
best_model = rf
best_mse = mse_random_forest